# Consultas anidadas en un caso de uso
### **Ingeniería de datos**
**Profesor: Domagoj Vrgoč**

### Introducción

En esta clase continuaremos con nuestro análisis de la base de datos de actores y peliculas. Para esto ocuparemos consultas anidadas y relacionadas.

### Requisitos

Para esta actividad, así como en todas las actividades de SQL, vamos a utilizar *Google colab* (https://colab.research.google.com), que es un entorno virtual permitiendo armar un servidor de bases de datos, y conectarse con este servidor. Para la conexión ocuparemos la herramienta llamada Jupyter Notebooks. Esta herramienta permite conectarse con un servidor SQL de la misma manera cómo hacerlo a través de la consola en un servidor local. 

El motor de bases de datos que ocuparemos en este curso se llama PostgreSQL, y uno siempre puede instalarlo localmente en su computador. Idea de ocupar Google colab es saltarse este paso, y no tener problemas con instalar, habilitar, o correr un motor de bases de datos.

Por lo tanto, para una actividad de SQL, en este curso siempre ocuparemos Jupyter Notebooks con Google colab. Para esto, se les entregará un archivo con extensión .ipynb, cual hay que subir a la plataforma Google Colab. Al inicio del tutorial mostraremos cómo funciona este proceso.


### Esquema

Para esta actividad, de nuevo trabajaremos con el siguiente esquema:

- `Peliculas(pid, pnombre, paño, pcategoria, pcalificacion, pdirector)`

- `Actores(aid, anombre, aedad)`

- `actuo_en(aid, pid, rol)`

Este esquema corresponde a películas, actores, y la información de los roles interpretados por una actor en una película.

Las llaves en nuestro caso son:
1. `pid`, para `Peliculas`
2. `aid` para `Actores`
3. `(aid,pid,rol)` para `actuo_en`.

Es importante notar que en este caso `rol` forma parte de la llave primaria de la relación `actuo_en`, dado que debemos modelar el caso donde un actor juega dos roles distintos en una misma película.

## Tutorial

Lo primero que hay que hacer es subir este notebook a https://colab.research.google.com

### Iniciar el servidor

Para iniciar el servidor virtual, *instalar* la base de datos postgres debe correr el siguiente bloque:

In [ ]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# set connection
%load_ext sql
%config SqlMagic.feedback=False 
%config SqlMagic.autopandas=True
%sql postgresql+psycopg2://@/postgres

### Creando la base de datos

Para crear y poblar nuestra base de datos, corramo el siguiente bloque de código:

In [ ]:
%%sql

DROP TABLE IF EXISTS Peliculas;
DROP TABLE IF EXISTS Actores;
DROP TABLE IF EXISTS Actuo_En;

CREATE TABLE IF NOT EXISTS Peliculas(
    pid int PRIMARY KEY,
    pnombre varchar(30),
    paño int,
    pcategoria varchar(30),
    pcalificacion float,
    pdirector varchar(30)
);

CREATE TABLE Actores(
    aid int PRIMARY KEY,
    anombre varchar(30),
    aedad int
);

CREATE TABLE actuo_en(
    aid int,
    pid int,
    rol varchar(30),
    PRIMARY KEY (aid,pid,rol)
);

INSERT INTO Peliculas VALUES(1,'Avengers:Endgame',2019,'SciFi',8.4,'Brothers Russo');
INSERT INTO Peliculas VALUES(2,'Captain America: Civil War',2016,'SciFi',8.7,'Brothers Russo');
INSERT INTO Peliculas VALUES(3,'Iron Man',2008,'SciFi',9.1,'John Favreu');
INSERT INTO Peliculas VALUES(4,'Batman: The Dark Knight',2008,'Thriller',8.3,'Christoper Nolan');
INSERT INTO Peliculas VALUES(5,'Batman: The Dark Knight Rises',2012,'Thriller',7.9,'Christoper Nolan');
INSERT INTO Peliculas VALUES(6,'Interstellar',2014,'Drama',8.4,'Christoper Nolan');
INSERT INTO Peliculas VALUES(7,'Sherlock Holmes',2009,'Mystery',7.8,'Guy Ritchie');
INSERT INTO Peliculas VALUES(8,'Avengers: Age of Ultron',2015,'SciFi',8.3,'Joss Whedon');
INSERT INTO Peliculas VALUES(9,'Doctor Strange',2016,'SciFi',8.8,'Scott Derrickson');

INSERT INTO Actores VALUES(1,'Robert Downey Jr.',57);
INSERT INTO Actores VALUES(2,'Scarlett Johansson',37);
INSERT INTO Actores VALUES(3,'Chris Evans',40);
INSERT INTO Actores VALUES(4,'Christian Bale',48);
INSERT INTO Actores VALUES(5,'Anne Hathaway',39);
INSERT INTO Actores VALUES(6,'Paul Bettany',50);
INSERT INTO Actores VALUES(7,'Benedict Cumberbatch',45);

INSERT INTO actuo_en VALUES(1,1,'Tony Stark');
INSERT INTO actuo_en VALUES(1,2,'Tony Stark');
INSERT INTO actuo_en VALUES(1,3,'Tony Stark');
INSERT INTO actuo_en VALUES(1,7,'Sherlock Holmes');
INSERT INTO actuo_en VALUES(2,1,'Natasha Romanoff');
INSERT INTO actuo_en VALUES(2,2,'Natasha Romanoff');
INSERT INTO actuo_en VALUES(3,1,'Steve Rogers');
INSERT INTO actuo_en VALUES(3,2,'Steve Rogers');
INSERT INTO actuo_en VALUES(4,4,'Bruce Wayne');
INSERT INTO actuo_en VALUES(4,5,'Bruce Wayne');
INSERT INTO actuo_en VALUES(5,5,'Selina Kyle');
INSERT INTO actuo_en VALUES(5,6,'Amelia Brand');
INSERT INTO actuo_en VALUES(6,8,'J.A.R.V.I.S.');
INSERT INTO actuo_en VALUES(6,8,'Vision');
INSERT INTO actuo_en VALUES(7,9,'Doctor Strange');
INSERT INTO actuo_en VALUES(7,9,'Dormammu');
INSERT INTO actuo_en VALUES(1,8,'Tony Stark');
INSERT INTO actuo_en VALUES(3,8,'Steve Rogers');
INSERT INTO actuo_en VALUES(2,8,'Natasha Romanoff');
INSERT INTO actuo_en VALUES(6,3,'J.A.R.V.I.S.');


Ahora podemos visualizar el contenido de las tablas:

In [ ]:
%%sql

SELECT * FROM Peliculas;

In [ ]:
%%sql

SELECT * FROM Actores;

In [ ]:
%%sql

SELECT * FROM actuo_en;

### Analizando la base de datos con consultas anidadas

Ahora que nos recordamos de los contenidos de nuestra base de datos, empezaremos analizarlos con consultas anidadas.

### Pregunta 1

Primero nos enfocaremos en la tabla `Películas` para conseguir la película de la categoría `SciFi` con la mayor calificación. Para esto, primero queremos encontrar cual es la mayor calificación de una película `SciFi`, lo que podemos hacer con una simple consulta de agregación:

In [ ]:
%%sql

SELECT MAX(pcalificacion)
FROM Peliculas
WHERE pcategoria = 'SciFi';

Para conseguir la película misma, ahora podemos ocupar el resultado de esta consulta en la claúsula `WHERE`:

In [ ]:
%%sql

SELECT P1.pnombre
FROM Peliculas AS P1
WHERE P1.pcategoria = 'SciFi' AND 
      P1.pcalificacion = (
                          SELECT MAX(P2.pcalificacion)
                          FROM Peliculas AS P2
                          WHERE P2.pcategoria = 'SciFi'
                         )

Noten que en la clusula `WHERE` también tenemos que validar que la película es de la categoría `SciFi`, en el caso que hay películas de distinta categoría con la misma calificación.

### Pregunta 2

Subconsulta podemos combinar con joins también. Por ejemplo, si nos interesan los nombres de todos los actores actuando en la película con la mayor calificación, podemos escribir:

In [ ]:
%%sql

SELECT Actores.anombre
FROM Peliculas AS P1, Actores, Actuo_En
WHERE P1.pid = Actuo_En.pid AND 
      Actuo_En.aid = Actores.aid AND
      P1.pcalificacion = (
                          SELECT MAX(P2.pcalificacion)
                          FROM Peliculas AS P2
                         )

Aquí la primera parte determina cual actor actua en cual película, y compara la calificación de cada película con la calificación máxima, obtenida con la consulta anidada.

### Pregunta 3

Una consulta puede tener más de una subconsulta anidada. Por ejemplo, si nos interesa el nombre de los actores que actuan en la película `SciFi` mejor calificada, y en la película de la categoría `Mystery` peor calificada, podemos primero computar la película `SciFi` mejor calificada, y la película `Mystery` peor calificada. La mejor película(s) `SciFi` ya computamos arriba. Para la peor película(s) `Mystery` ocuparemos:

In [ ]:
%%sql

SELECT P1.pnombre
FROM Peliculas AS P1
WHERE P1.pcategoria = 'Mystery' AND 
      P1.pcalificacion = (
                          SELECT MIN(P2.pcalificacion)
                          FROM Peliculas AS P2
                          WHERE P2.pcategoria = 'Mystery'
                         )

Ahora estas dos tablas podemos ocupar en la claúsula `WHERE`:

In [ ]:
%%sql

SELECT Actores.anombre
FROM Actores, Actuo_En AS AE1, Actuo_En AS AE2
WHERE Actores.aid = AE1.aid AND AE1.pid IN (SELECT P1.pid
                                            FROM Peliculas AS P1
                                            WHERE P1.pcategoria = 'SciFi' AND 
                                            P1.pcalificacion = (
                                                              SELECT MAX(P2.pcalificacion)
                                                              FROM Peliculas AS P2
                                                              WHERE P2.pcategoria = 'SciFi'
                                                              )) AND
      Actores.aid = AE2.aid AND AE2.pid IN (
                                            SELECT P3.pid
                                            FROM Peliculas AS P3
                                            WHERE P3.pcategoria = 'Mystery' AND 
                                            P3.pcalificacion = (
                                                              SELECT MIN(P4.pcalificacion)
                                                              FROM Peliculas AS P4
                                                              WHERE P4.pcategoria = 'Mystery'
                                                               ))

Para validar que un actor actua en dos películas, debemos hacer un self-join de la tabla `Actuo_En`. Después, la parte `AE1` valida si el actor actua en la película `SciFi` con la mayor calificación, y la parte `AE2` si el actor actua en la película `Mystery` con la menor calificación.

### Pregunta 4

Consideremos de nuevo la tabla `Pelicuas`. Ahora queremos, para cada categoría de la película, listar solo las películas cuya calificación es mayor que el promedia de las calificaciónes de las películas de su genero. Para esto, primero computaremos el promedio de calificaciones por categoría.

In [ ]:
%%sql

SELECT Peliculas.pcategoria, AVG(Peliculas.pcalificacion)
FROM Peliculas
GROUP BY Peliculas.pcategoria;

Ahora ocuparemos una simplifiacion de esta consulta cómo una subconsulta para nuestro resultado final. Lo distinto aquí será que necesitaremos ocupar consultas relacionadas:

In [ ]:
%%sql

SELECT P1.pcategoria, P1.pnombre
FROM Peliculas AS P1
WHERE P1.pcalificacion >= (
                          SELECT AVG(P2.pcalificacion)
                          FROM Peliculas AS P2
                          WHERE P2.pcategoria = P1.pcategoria
                          )
ORDER BY P1.pcategoria;

Lo qué ocurre aquí es que al calcular el promedio en la consulta anidada, consideraremos solo las películas de la misma categoría cómo la película `P1` de la consulta exterior. Finalmente, los resultados los entregamos ordenados por categoría de la película.

### Pregunta 5

Considere ahora la consulta que quiere listar todas las películas del actor que actua en la mayor cantidad de películas en nuestra base de datos. Para esto, primero queremos calcular el número de películas por actor:

In [ ]:
%%sql

SELECT Actores.anombre, COUNT(DISTINCT actuo_en.pid) AS npeliculas
FROM actuo_en, Actores
WHERE actuo_en.aid = Actores.aid
GROUP BY actuo_en.aid, Actores.anombre

Para encontrar el máximo, podemos ocupar la siguiente consulta anidada:

In [ ]:
%%sql

SELECT MAX(ActorPelicula.npeliculas)
FROM (
    SELECT Actores.anombre, COUNT(DISTINCT actuo_en.pid) AS npeliculas
    FROM actuo_en, Actores
    WHERE actuo_en.aid = Actores.aid
    GROUP BY actuo_en.aid, Actores.anombre
) AS ActorPelicula

Finalmente, podemos ocupar la siguiente consulta con dos consultas anidadas para recuperar el nombre del actor:

In [ ]:
%%sql

SELECT AP1.anombre
FROM (
    SELECT Actores.anombre, COUNT(DISTINCT actuo_en.pid) AS npeliculas
    FROM actuo_en, Actores
    WHERE actuo_en.aid = Actores.aid
    GROUP BY actuo_en.aid, Actores.anombre
) AS AP1
WHERE AP1.npeliculas = (
                        SELECT MAX(AP2.npeliculas)
                        FROM (
                              SELECT Actores.anombre, COUNT(DISTINCT actuo_en.pid) AS npeliculas
                              FROM actuo_en, Actores
                              WHERE actuo_en.aid = Actores.aid
                              GROUP BY actuo_en.aid, Actores.anombre
                              ) AS AP2
                        )

Aquí la primera consulta anidada, llamada `AP1`, contiene la información de número de películas por actor. Después la segunda consulta anidada, `AP2`, revisa que el número de películas de `AP1` es igual a máximo número de películas.

Noten que no es suficiente solo ordenar los resultados de `AP1` y quedanos con la primera fila, dado que pueden existir dos o más actores con el mismo máximo número de películas. Para ilustrar esto, si cambiamos la consulta, y queremos devolver los nombres de los actores con más películas que el promedio de películas por actor, ahora tenemos:

In [ ]:
%%sql

SELECT AP1.anombre
FROM (
    SELECT Actores.anombre, COUNT(DISTINCT actuo_en.pid) AS npeliculas
    FROM actuo_en, Actores
    WHERE actuo_en.aid = Actores.aid
    GROUP BY actuo_en.aid, Actores.anombre
) AS AP1
WHERE AP1.npeliculas >= (
                        SELECT AVG(AP2.npeliculas)
                        FROM (
                              SELECT Actores.anombre, COUNT(DISTINCT actuo_en.pid) AS npeliculas
                              FROM actuo_en, Actores
                              WHERE actuo_en.aid = Actores.aid
                              GROUP BY actuo_en.aid, Actores.anombre
                              ) AS AP2
                        )

### Resumen

En este tutorial aprendimos cómo analizar una base de datos de películas ocupando consultas anidadas, combinandolas con otras funcionalidades de SQL aprendidas a lo largo del curso.